# Evaluation

In [1]:
import os
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [101]:
@dataclass(frozen=True)
class EvaluationConfig:
    test_data: Path
    #scores: Path

In [102]:
from CementStrength.utils import read_yaml,create_directories
#from CementStrength.entity import DataIngestionConfig,DataValidationConfig,DbOperationsConfig,TrainingConfig,EvaluationConfig
from CementStrength.constants import *





class ConfigurationManager:
    
    def __init__(self, 
        config_file= CONFIG_FILE_PATH,
        param_file = PARAMS_FILE_PATH):

        self.config_file = read_yaml(config_file)
        self.param_file = read_yaml(param_file)

    def get_evaluation_config(self) -> EvaluationConfig:
                       
        evaluation_config = EvaluationConfig(
                  test_data= self.config_file.test_data,
                  #scores= self.config_file.scores
                  
                           
                  )
        return evaluation_config

In [103]:
from CementStrength.utils import load_model
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score


class Evaluation:

    def __init__(self,config: EvaluationConfig):
        self.config = config

    def evaluate(self):
        X_test = pd.read_csv(os.path.join(self.config.test_data,'test_data_featues.csv'))
        y_test = pd.read_csv(os.path.join(self.config.test_data,'test_data_labels.csv'))
        scaler = StandardScaler()
        X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test),columns=X_test.columns,index=X_test.index)
        cluster_model = load_model('cluster_model',os.path.join('Training','models'))
        prediction_cluster_model = cluster_model.predict(X_test_scaled)
        X_test_scaled['cluster'] = pd.Series(prediction_cluster_model, index=X_test_scaled.index) #cluster column added
        dfs=[X_test_scaled[X_test_scaled['cluster']==i] for i in range(len(set(prediction_cluster_model)))]
        for i in range(len(dfs)):
            dfs[i] = dfs[i].drop(columns='cluster')
        y_clusterwise = [y_test.loc[dfs[i].index] for i in range(len(dfs))]
        cluster0_model = load_model('random_model_cluster0',os.path.join('Training','models'))
        cluster1_model = load_model('random_model_cluster1',os.path.join('Training','models'))
        cluster2_model = load_model('random_model_cluster2',os.path.join('Training','models'))
        cluster3_model = load_model('random_model_cluster3',os.path.join('Training','models'))
        prediction_cluster_0 = cluster0_model.predict(dfs[0])
        score_cluster_0 = r2_score(y_clusterwise[0],prediction_cluster_0)
        prediction_cluster_1 = cluster1_model.predict(dfs[1])
        score_cluster_1 = r2_score(y_clusterwise[1],prediction_cluster_1)
        prediction_cluster_2 = cluster2_model.predict(dfs[2])
        score_cluster_2 = r2_score(y_clusterwise[2],prediction_cluster_2)
        prediction_cluster_3 = cluster3_model.predict(dfs[3])
        score_cluster_3 = r2_score(y_clusterwise[3],prediction_cluster_3)
        scores = {'r2_score0':score_cluster_0,'r2_score1':score_cluster_1,'r2_score2':score_cluster_2,'r2_score3':score_cluster_3}
        save_json(path=Path("scores.json"),data=scores)
        
        

In [99]:
from CementStrength.utils import save_json

ImportError: cannot import name 'save_json' from 'CementStrength.utils' (c:\users\gagan\documents\projects\cementstrength\src\CementStrength\utils\__init__.py)

import json
def save_json(path: Path, data: dict):
    with open(path, "w") as f:
        json.dump(data, f, indent=4)

    #logger.info(f"json file saved at: {path}")

In [104]:
configs = ConfigurationManager()
eval_configs =configs.get_evaluation_config()
eval_ob = Evaluation()
eval_ob.evaluate()

[2023-11-17 12:46:22,008: INFO: common: yaml file: config\config.yml loaded successfully]
[2023-11-17 12:46:22,010: INFO: common: yaml file: params.yml loaded successfully]


BoxKeyError: "'Box' object has no attribute 'test_data'"

In [9]:
cluster_model = load_model('cluster_model',os.path.join('Training','models'))

In [10]:
config= read_yaml('config/config.yml')

[2023-11-17 10:38:12,430: INFO: common: yaml file: config/config.yml loaded successfully]


In [21]:
X_test = pd.read_csv(Path(os.path.join(config.evaluation.test_data,'test_data_features.csv')))
scaler = StandardScaler()
X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test),columns=X_test.columns,index=X_test.index)

In [20]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [22]:
pred = cluster_model.predict(X_test_scaled)

c:\Users\gagan\Anaconda3\envs\cementnew\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


In [42]:
X_test_scaled['cluster'] = pd.Series(pred, index=X_test_scaled.index) #cluster column added
dfs=[X_test_scaled[X_test_scaled['cluster']==i] for i in range(len(set(pred)))]
for i in range(len(dfs)):
    dfs[i] = dfs[i].drop(columns='cluster')

In [33]:
y_test = pd.read_csv(os.path.join(config.evaluation.test_data,'test_data_labels.csv'))
y_clusterwise = [y_test.loc[dfs[i].index] for i in range(len(dfs))]

In [75]:
y_clusterwise[3].head()

,Concrete_compressive _strength
0,3.877017
13,3.538057
16,4.360548
24,3.873282
28,3.812865


In [76]:
dfs[3].head()

,Cement _component_1,Blast Furnace Slag _component_2,Fly Ash _component_3,Water_component_4,Superplasticizer_component_5,Coarse Aggregate_component_6,Fine Aggregate_component_7,Age_day
0,0.657388,0.079498,1.266035,-0.687001,1.306533,0.315173,-1.561610,0.315827
13,0.542398,1.085835,-0.880996,-1.357701,1.118830,-1.609514,1.338583,-1.071722
16,0.882137,1.037855,-0.880996,-0.808163,0.946364,-0.337099,-0.135159,1.043893
24,1.002385,0.946585,-0.880996,-1.468986,1.207180,1.919158,-2.283888,-1.071722
28,0.165930,0.838366,1.019450,0.009608,0.732144,-1.351693,0.023246,0.315827


In [38]:
cluster0_model = load_model('random_model_cluster0',os.path.join('Training','models'))
cluster1_model = load_model('random_model_cluster1',os.path.join('Training','models'))
cluster2_model = load_model('random_model_cluster2',os.path.join('Training','models'))
cluster3_model = load_model('random_model_cluster3',os.path.join('Training','models'))

In [39]:
from sklearn.metrics import r2_score

In [64]:
prediction_cluster_1 = cluster1_model.predict(dfs[1])

In [65]:
score_cluster_3 = r2_score(y_clusterwise[1],prediction_cluster_1)

In [66]:
score_cluster_3

0.8080273445506383

In [67]:
prediction_cluster_0 = cluster0_model.predict(dfs[0])
score_cluster_0 = r2_score(y_clusterwise[0],prediction_cluster_0)
prediction_cluster_1 = cluster1_model.predict(dfs[1])
score_cluster_1 = r2_score(y_clusterwise[1],prediction_cluster_1)
prediction_cluster_2 = cluster2_model.predict(dfs[2])
score_cluster_2 = r2_score(y_clusterwise[2],prediction_cluster_2)
prediction_cluster_3 = cluster3_model.predict(dfs[3])
score_cluster_3 = r2_score(y_clusterwise[3],prediction_cluster_3)
    

In [68]:
score_cluster_0

0.8307809215060173

In [69]:
score_cluster_1

0.8080273445506383

In [70]:
score_cluster_2

0.9155376681551859

In [71]:
score_cluster_3

0.4224113436937028

In [81]:
dic = {'score0':score_cluster_0,'score1':score_cluster_1,'score2':score_cluster_2,'score3':score_cluster_3}

In [82]:
dic

{'score0': 0.8307809215060173,
 'score1': 0.8080273445506383,
 'score2': 0.9155376681551859,
 'score3': 0.4224113436937028}

In [83]:
import json

In [105]:
from CementStrength.utils import load_model

In [106]:
save_json(path=Path("scores.json"),data=dic)